# Day 19
## Part 1
~~I'm not bothering with this one.~~ Ok, the single empty day is nagging at me. Let's take this one slowly and not spend entire days debugging it.

The idea is for each scanner to calculate the absolute differences on each axis between each detected pair of beacons, forming a matrix of three differences for each pair. Represent this as a graph. Then, for rach pair of scanners, calculate the overlapping distances. The pair of scanners with the highest number of mutual distances can combine their beacon maps into one by manipulating the axes of one to match the other. Repeat until their is only one beacon map left.

Parse the scanners as lists of three-coordinate tuples.

In [24]:
import parse

def parse_data(s):
    scanners = []
    for block in s.strip().split('\n\n'):
        scanner = []
        for line in block.splitlines():
            if r := parse.parse('{x:d},{y:d},{z:d}', line):
                scanner.append((r['x'], r['y'], r['z']))
        scanners.append(scanner)
    return scanners

test_data = parse_data(open('test_input', 'r').read())
data = parse_data(open('input', 'r').read())

Record the distance between each scanner's observations as a set of absolute differences in each coordinate direction, which will allow comparison with other scanners' coordinate systems.

In [2]:
import itertools
import networkx as nx

def beacon_map(scanner):
    distances = nx.Graph()
    for (i, (x_i, y_i, z_i)), (j, (x_j, y_j, z_j)) in itertools.combinations(enumerate(scanner), 2):
        distance = frozenset([abs(d) for d in [x_i - x_j, y_i - y_j, z_i - z_j]])
        distances.add_edge(i, j, distance=distance)
        distances.nodes[i]['coordinate'] = (x_i, y_i, z_i)
        distances.nodes[j]['coordinate'] = (x_j, y_j, z_j)
    return distances

test_beacon_distances = [beacon_map(x) for x in test_data]
beacon_distances = [beacon_map(x) for x in data]

Put each of the distances into a set for each scanner.

In [3]:
def distances_in_graphs(beacon_maps):
    result = []
    for b in beacon_maps:
        result.append({b.edges[i, j]['distance'] for i, j in b.edges})
    return result

test_set_distances = distances_in_graphs(test_beacon_distances)
set_distances = distances_in_graphs(beacon_distances)

Count the mutual distances between them.

In [4]:
def count_mutual_distances(set_distances):
    result = []
    for (i, sdi), (j, sdj) in itertools.combinations(enumerate(set_distances), 2):
        result.append((i, j, len(sdi & sdj)))
    return(result)

test_mutual_distances = count_mutual_distances(test_set_distances)
mutual_distances = count_mutual_distances(set_distances)

Find the ids of the scanners with the highest number of mutual distances.

In [25]:
def most_overlapping_scanner_pair(mutual_distances):
    i, j, _ = max(mutual_distances, key=lambda x: x[2])
    return i, j
    
test_most_overlapping = most_overlapping_scanner_pair(test_mutual_distances)
test_most_overlapping

(0, 1)

For these scanners find the distances from each beacon to other beacons. Then compare between the two sets to match nodes.

In [27]:
def common_distances(beacons_1, beacons_2):
    return ({beacons_1.edges[edge]['distance'] for edge in beacons_1.edges}
            & {beacons_2.edges[edge]['distance'] for edge in beacons_2.edges})

def node_distances(beacons, distances):
    return {
        node: {
            beacons[node][nbr]['distance'] 
            for nbr in beacons[node]
            if beacons[node][nbr]['distance'] in distances
        } 
        for node in beacons
    }

test_common_distances = common_distances(test_beacon_distances[0], test_beacon_distances[1])
test_node_distances_0 = node_distances(test_beacon_distances[0], test_common_distances)
test_node_distances_1 = node_distances(test_beacon_distances[1], test_common_distances)

In [29]:
test_node_distances_0

{0: {frozenset({277, 749, 1282}),
  frozenset({11, 39, 140}),
  frozenset({231, 889, 1248}),
  frozenset({236, 280, 1022}),
  frozenset({235, 443, 941}),
  frozenset({228, 326, 1065}),
  frozenset({259, 851, 1219}),
  frozenset({14, 87, 108}),
  frozenset({19, 113, 1335}),
  frozenset({55, 119, 1302}),
  frozenset({55, 124, 1310})},
 1: {frozenset({173, 984, 1189}),
  frozenset({180, 867, 1065}),
  frozenset({8, 64, 69}),
  frozenset({16, 1299}),
  frozenset({32, 138, 1202}),
  frozenset({91, 314, 975}),
  frozenset({25, 58, 105}),
  frozenset({28, 332, 873}),
  frozenset({62, 286, 1013}),
  frozenset({55, 124, 1310}),
  frozenset({181, 1030, 1146})},
 2: set(),
 3: {frozenset({26, 33, 1227}),
  frozenset({346, 837, 1111}),
  frozenset({149, 172, 1008}),
  frozenset({364, 735, 1174}),
  frozenset({141, 218, 1051}),
  frozenset({148, 335, 927}),
  frozenset({32, 138, 1202}),
  frozenset({318, 875, 1140}),
  frozenset({14, 87, 108}),
  frozenset({48, 97, 154}),
  frozenset({32, 69, 1194}

In [30]:
test_node_distances_1

{0: {frozenset({123, 1041, 1055}),
  frozenset({1, 81, 163}),
  frozenset({8, 43, 46}),
  frozenset({171, 495, 939}),
  frozenset({149, 172, 1008}),
  frozenset({273, 513, 1002}),
  frozenset({197, 269, 1162}),
  frozenset({117, 1022, 1077}),
  frozenset({133, 467, 968}),
  frozenset({236, 280, 1022}),
  frozenset({181, 1030, 1146})},
 1: {frozenset({1, 81, 163}),
  frozenset({235, 443, 941}),
  frozenset({196, 432, 1081}),
  frozenset({90, 494, 776}),
  frozenset({116, 859, 996}),
  frozenset({192, 512, 839}),
  frozenset({180, 867, 1065}),
  frozenset({122, 892, 960}),
  frozenset({148, 335, 927}),
  frozenset({52, 466, 805}),
  frozenset({7, 117, 124})},
 2: {frozenset({214, 487, 893}),
  frozenset({28, 29, 38}),
  frozenset({90, 494, 776}),
  frozenset({171, 495, 939}),
  frozenset({346, 837, 1111}),
  frozenset({83, 378, 906}),
  frozenset({18, 63, 102}),
  frozenset({259, 851, 1219}),
  frozenset({91, 314, 975}),
  frozenset({116, 372, 870}),
  frozenset({298, 991, 1208})},
 3: {

The node distances can then be matched to determine which beacons are the same within the different coordinate systems.

In [ ]:
def match_nodes(node_distances_1, node_distances_2):
    possible_matches = {}
    for node_1, node_2 in itertools.product(node_distances_1, node_distances_2):
        overlap = node_distances_1[node_1] & node_distances_2[node_2]
        if len(overlap) > 1: 
            possible_matches[(node_1, node_2)] = len(overlap)
    return possible_matches